In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10687
2,Huelva,Confirmados PDIA 14 días,995
3,Huelva,Tasa PDIA 14 días,"194,81422163397247"
4,Huelva,Confirmados PDIA 7 días,427
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,123
630,Municipio de Huelva sin especificar,Curados,100


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10687.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3015.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1168 personas en los últimos 7 días 

Un positivo PCR cada 495 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10687.0,427.0,995.0,510743.0,83.603691,194.814222,25.0
Huelva-Costa,5990.0,268.0,652.0,288115.0,93.018413,226.298527,24.0
Huelva (capital),3015.0,123.0,290.0,143663.0,85.617034,201.861300,9.0
Aljaraque,414.0,25.0,52.0,21260.0,117.591722,244.590781,8.0
Jabugo,46.0,11.0,12.0,2250.0,488.888889,533.333333,5.0
Sierra de Huelva-Andévalo Central,1283.0,20.0,40.0,67571.0,29.598496,59.196993,5.0
Cartaya,374.0,26.0,64.0,19974.0,130.169220,320.416542,4.0
Isla Cristina,408.0,33.0,106.0,21264.0,155.191874,498.495109,3.0
Moguer,362.0,25.0,61.0,22088.0,113.183629,276.168055,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),518.0,38.0,81.0,10761.0,353.127033,752.718149,1.0
Lucena del Puerto,93.0,10.0,18.0,3371.0,296.647879,533.966182,0.0
Jabugo,46.0,11.0,12.0,2250.0,488.888889,533.333333,5.0
Isla Cristina,408.0,33.0,106.0,21264.0,155.191874,498.495109,3.0
Paterna del Campo,64.0,4.0,12.0,3509.0,113.992590,341.977771,0.0
Lepe,940.0,38.0,93.0,27431.0,138.529401,339.032481,0.0
Cartaya,374.0,26.0,64.0,19974.0,130.169220,320.416542,4.0
Villalba del Alcor,103.0,1.0,10.0,3338.0,29.958059,299.580587,0.0
San Bartolomé de la Torre,45.0,1.0,11.0,3736.0,26.766595,294.432548,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,45.0,1.0,11.0,3736.0,26.766595,294.432548,0.0,0.090909
Villalba del Alcor,103.0,1.0,10.0,3338.0,29.958059,299.580587,0.0,0.100000
Gibraleón,349.0,2.0,11.0,12607.0,15.864202,87.253113,-2.0,0.181818
Rociana del Condado,116.0,1.0,5.0,7855.0,12.730745,63.653724,0.0,0.200000
Minas de Riotinto,67.0,1.0,4.0,3848.0,25.987526,103.950104,0.0,0.250000
Valverde del Camino,273.0,2.0,8.0,12820.0,15.600624,62.402496,0.0,0.250000
Beas,57.0,1.0,4.0,4257.0,23.490721,93.962885,0.0,0.250000
Isla Cristina,408.0,33.0,106.0,21264.0,155.191874,498.495109,3.0,0.311321
Zalamea la Real,47.0,1.0,3.0,3068.0,32.594524,97.783572,-1.0,0.333333
